# YOLOv10 Custom training
Jupyter notebook used to train custom model on Google CoLab for specific application


In [14]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/'

#Change directory to Google Drive Direcotory
os.chdir(f'{DRIVE_PATH}')

#Set as Home directory
HOME = os.getcwd()
print(HOME)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [15]:
#Define Session
SESSION_NAME    = 'OpenImageTest'
MODEL_PATH      = f'{HOME}/weights/yolov10n.pt'

#Dataset Variablesh
classes = ['Horse','Sheep','Cattle']

#Training Variables
EPOCHS = 30

#Validation Variables
number_mapping = {
    '0': '17',      # Dataset ID : COCO ID
    '1': '18',
    '2': '19',
}

#Add Dataset Paths

In [16]:
DATASETS_DIR          = f'{HOME}/datasets'
DATASET_DIR           = f'{DATASETS_DIR}/OpenImages-LiveStock'
DATASET_YAML_PATH     = f'{DATASET_DIR}/dataset.yaml'

## Enable GPU acceleration
Navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

In [17]:
#Check if GPU is active
!nvidia-smi

Sun Aug 25 12:17:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  70W |    901MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install YOLOv10

In [18]:
# Clone YOLOv10 from Github
!pip install -q git+https://github.com/THU-MIG/yolov10.git

# Download pretrained weights
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
total 55M
-rw------- 1 root root 11M May 26 15:54 yolov10n.pt
-rw------- 1 root root 11M May 26 15:54 yolov10n.pt.1
-rw------- 1 root root 11M May 26 15:54 yolov10n.pt.2
-rw------- 1 root root 11M May 26 15:54 yolov10n.pt.3
-rw------- 1 root root 11M May 26 15:54 yolov10n.pt.4


## Custom Training

In [ ]:
%cd {HOME}

from datetime import datetime
from ultralytics import YOLOv10

#Session Variables
T_STAMP         = datetime.now().strftime('%Y%m%d-%H:%M')
RUN_NAME        = SESSION_NAME + '-' + T_STAMP

#Create Output Path
RUNS_PATH       = os.path.join(HOME, 'runs')
SESSION_PATH    = os.path.join(RUNS_PATH, SESSION_NAME)
OUTPUT_PATH     = os.path.join(SESSION_PATH, RUN_NAME)

#Create YOLOv10 model
model           = YOLOv10(MODEL_PATH)

#Train the model
train = model.train(
        #Train Settings
        model=MODEL_PATH, 	          #Specifies the model file for training.
        data=DATASET_YAML_PATH,       #Path to the dataset configuration file (e.g., coco8.yaml).
        epochs=EPOCHS, 	                  #Total number of training epochs.
        time=None, 	                  #Maximum training time in hours.
        patience=100, 	              #Number of epochs to wait without improvement in validation metrics before early stopping the training.
        batch=-1, 	                  #Batch size, with three modes: set as an integer (e.g., batch=16), auto mode for 60% GPU memory utilization (batch=-1), or auto mode with specified utilization fraction (batch=0.70).
        imgsz=640, 	                  #Target image size for training.

        save=True, 	                  #Enables saving of training checkpoints and final model weights.
        save_period=-1, 	            #Frequency of saving model checkpoints, specified in epochs.
        cache=False, 	                #Enables caching of dataset images in memory (True/ram), on disk (disk), or disables it (False).
        device=None, 	                  #Specifies the computational device(s) for training
        workers=8, 	                  #Number of worker threads for data loading (per RANK if Multi-GPU training).
        project=SESSION_PATH, 	      #Name of the project directory where training outputs are saved.
        name=RUN_NAME, 	              #Name of the training run.
        exist_ok=False, 	            #If True, allows overwriting of an existing project/name directory.
        pretrained=True,	            #Determines whether to start training from a pretrained model.

        optimizer='auto', 	          #Choice of optimizer for training.
        verbose=False, 	              #Enables verbose output during training, providing detailed logs and progress updates.
        seed=0, 	                    #Sets the random seed for training, ensuring reproducibility of results across runs with the same configurations.
        deterministic=True, 	        #Forces deterministic algorithm use, ensuring reproducibility but may affect performance and speed due to the restriction on non-deterministic algorithms.
        single_cls=False, 	          #Treats all classes in multi-class datasets as a single class during training.
        rect=False, 	                #Enables rectangular training, optimizing batch composition for minimal padding. Can improve efficiency and speed but may affect model accuracy.
        cos_lr=False,	                #Utilizes a cosine learning rate scheduler, adjusting the learning rate following a cosine curve over epochs. Helps in managing learning rate for better convergence.
        close_mosaic=10, 	            #Disables mosaic data augmentation in the last N epochs to stabilize training before completion. Setting to 0 disables this feature.
        resume=False, 	              #Resumes training from the last saved checkpoint. Automatically loads model weights, optimizer state, and epoch count, continuing training seamlessly.
        amp=True, 	                  #Enables Automatic Mixed Precision (AMP) training, reducing memory usage and possibly speeding up training with minimal impact on accuracy.
        fraction=1.0, 	              #Specifies the fraction of the dataset to use for training. Allows for training on a subset of the full dataset, useful for experiments or when resources are limited.
        profile=False, 	              #Enables profiling of ONNX and TensorRT speeds during training, useful for optimizing model deployment.

        freeze=None,          	      #Freezes the first N layers of the model or specified layers by index, reducing the number of trainable parameters. Useful for fine-tuning or transfer learning.
        lr0=0.01, 	                  #Initial learning rate (i.e. SGD=1E-2, Adam=1E-3) . Adjusting this value is crucial for the optimization process, influencing how rapidly model weights are updated.
        lrf=0.1, 	                    #Final learning rate as a fraction of the initial rate = (lr0 * lrf), used in conjunction with schedulers to adjust the learning rate over time.
        momentum=0.937, 	            #Momentum factor for SGD or beta1 for Adam optimizers, influencing the incorporation of past gradients in the current update.
        weight_decay=0.0005, 	        #L2 regularization term, penalizing large weights to prevent overfitting.
        warmup_epochs=3.0, 	          #Number of epochs for learning rate warmup, gradually increasing the learning rate from a low value to the initial learning rate to stabilize training early on.
        warmup_momentum=0.8, 	        #Initial momentum for warmup phase, gradually adjusting to the set momentum over the warmup period.
        warmup_bias_lr=0.1, 	        #Learning rate for bias parameters during the warmup phase, helping stabilize model training in the initial epochs.

        box=7.5, 	                    #Weight of the box loss component in the loss function, influencing how much emphasis is placed on accurately predicting bounding box coordinates.
        cls=0.5, 	                    #Weight of the classification loss in the total loss function, affecting the importance of correct class prediction relative to other components.
        dfl=1.5, 	                    #Weight of the distribution focal loss, used in certain YOLO versions for fine-grained classification.
        pose=12.0, 	                  #Weight of the pose loss in models trained for pose estimation, influencing the emphasis on accurately predicting pose keypoints.
        kobj=2.0, 	                  #Weight of the keypoint objectness loss in pose estimation models, balancing detection confidence with pose accuracy.
        label_smoothing=0.0, 	        #Applies label smoothing, softening hard labels to a mix of the target label and a uniform distribution over labels, can improve generalization.
        nbs=64, 	                    #Nominal batch size for normalization of loss.

        overlap_mask=True, 	          #Determines whether segmentation masks should overlap during training, applicable in instance segmentation tasks.
        mask_ratio=4, 	              #Downsample ratio for segmentation masks, affecting the resolution of masks used during training.
        dropout=0.0, 	                #Dropout rate for regularization in classification tasks, preventing overfitting by randomly omitting units during training.
        val=True, 	                  #Enables validation during training, allowing for periodic evaluation of model performance on a separate dataset.
        plots=True,	                  #Generates and saves plots of training and validation metrics, as well as prediction examples, providing visual insights into model performance and learning progression.

        #Augmentation Arguments
        hsv_h=0.015,                  #Adjusts the hue of the image by a fraction of the color wheel, introducing color variability. Helps the model generalize across different lighting conditions.
        hsv_s=0.7,                    #Alters the saturation of the image by a fraction, affecting the intensity of colors. Useful for simulating different environmental conditions.
        hsv_v=0.4,                    #Modifies the value (brightness) of the image by a fraction, helping the model to perform well under various lighting conditions.
        degrees=0.0,                  #Rotates the image randomly within the specified degree range, improving the model's ability to recognize objects at various orientations.
        translate=0.1,                #Translates the image horizontally and vertically by a fraction of the image size, aiding in learning to detect partially visible objects.

        scale=0.5,                    #Scales the image by a gain factor, simulating objects at different distances from the camera.
        shear=0.0,                    #Shears the image by a specified degree, mimicking the effect of objects being viewed from different angles.
        perspective=0.0,              #Applies a random perspective transformation to the image, enhancing the model's ability to understand objects in 3D space.
        flipud=0.0,                   #Flips the image upside down with the specified probability, increasing the data variability without affecting the object's characteristics.
        fliplr=0.5,                   #Flips the image left to right with the specified probability, useful for learning symmetrical objects and increasing dataset diversity.

        bgr=0.0, 	                    #Flips the image channels from RGB to BGR with the specified probability, useful for increasing robustness to incorrect channel ordering.
        mosaic=1.0, 	                #Combines four training images into one, simulating different scene compositions and object interactions. Highly effective for complex scene understanding.
        mixup=0.0, 	                  #Blends two images and their labels, creating a composite image. Enhances the model's ability to generalize by introducing label noise and visual variability.
        copy_paste=0.0,	              #Copies objects from one image and pastes them onto another, useful for increasing object instances and learning object occlusion.
        auto_augment='randaugment',   #Automatically applies a predefined augmentation policy (randaugment, autoaugment, augmix), optimizing for classification tasks by diversifying the visual features.
        erasing=0.4,                  #Randomly erases a portion of the image during classification training, encouraging the model to focus on less obvious features for recognition.
        crop_fraction=1.0,            #Crops the classification image to a fraction of its size to emphasize central features and adapt to object scales, reducing background distractions.
)

/content/drive/MyDrive
New https://pypi.org/project/ultralytics/8.2.81 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/weights/yolov10n.pt, data=/content/drive/MyDrive/datasets/OpenImages-LiveStock/dataset.yaml, epochs=30, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/runs/OpenImageTest, name=OpenImageTest-20240825-12:17, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=Tr

train: Scanning /content/drive/MyDrive/datasets/OpenImages-LiveStock/labels/train.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/local/lib/python3.10/dist-packages/albumentations/core/composition.py:161: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/datasets/OpenImages-LiveStock/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/runs/OpenImageTest/OpenImageTest-20240825-12:17/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.000421875), 107 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/runs/OpenImageTest/OpenImageTest-20240825-12:17
Starting training for 30 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


...


      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


      30/30      6.06G      1.303      1.236      1.377      1.398      1.535       1.33          1        640: 100%|██████████| 38/38 [00:37<00:00,  1.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]

                   all        200        540      0.687       0.23      0.184      0.102



30 epochs completed in 0.385 hours.
Optimizer stripped from /content/drive/MyDrive/runs/OpenImageTest/OpenImageTest-20240825-12:17/weights/last.pt, 5.8MB
Optimizer stripped from /content/drive/MyDrive/runs/OpenImageTest/OpenImageTest-20240825-12:17/weights/best.pt, 5.8MB

Validating /content/drive/MyDrive/runs/OpenImageTest/OpenImageTest-20240825-12:17/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10n summary (fused): 285 layers, 2695976 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:02<00:02,  1.10s/it]

# Display Training Information

In [1]:
%cd {HOME}

#Display Confusion matrix
from IPython.display import Image
Image(filename=f'{OUTPUT_PATH}/confusion_matrix.png', width=600)
Image(filename=f'{OUTPUT_PATH}/results.png', width=600)

[Errno 2] No such file or directory: '{HOME}'
/content


NameError: name 'OUTPUT_PATH' is not defined

# Compare Pre-trained and Trained Model

## Prepare Dataset for validation
We need to have a version of the matches the coco dataset on which the pre-trained model was trained on. For example: The custom dataset has one class of 'sheep' at index '0'. But the pre-trained model was trained on 80 classes where 'person' has index '0', and sheep has index '18'. With the validation dataset, the `/labels/` directory has the actual class stored.

The pretrained model will go through the imageset. It will detect an object of index '18'. The (unedited) validation set will say the object belongs to class of index '0' and therefore an inaccurate detection. Thus, the original dataset needs to be converted the the coco class index. All the sheep instances of index '0', should now be changed to index '18'.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Create Dataset folder
COCO_DATASET_DIR = '/content/COCO_dataset'
COCO_DATASET_YAML_PATH = COCO_DATASET_DIR + '/dataset.yaml'
print(COCO_DATASET_YAML_PATH)
print(DATASET_DIR)
#Create a copy
!rm -rf {COCO_DATASET_DIR}
!cp -r {DATASET_DIR} {COCO_DATASET_DIR}

#Remove all *.cache files
%cd {COCO_DATASET_DIR}
!rm -r train/*.cache valid/*.cache test/*.cache
%cd {HOME}

In [ ]:
#Create new YAMl file to describe COCO classes for pre-trained model
COCO_CLASS_list = [
    "0: person",
    "1: bicycle",
    "2: car",
    "3: motorcycle",
    "4: airplane",
    "5: bus",
    "6: train",
    "7: truck",
    "8: boat",
    "9: traffic light",
    "10: fire hydrant",
    "11: stop sign",
    "12: parking meter",
    "13: bench",
    "14: bird",
    "15: cat",
    "16: dog",
    "17: horse",
    "18: sheep",
    "19: cow",
    "20: elephant",
    "21: bear",
    "22: zebra",
    "23: giraffe",
    "24: backpack",
    "25: umbrella",
    "26: handbag",
    "27: tie",
    "28: suitcase",
    "29: frisbee",
    "30: skis",
    "31: snowboard",
    "32: sports ball",
    "33: kite",
    "34: baseball bat",
    "35: baseball glove",
    "36: skateboard",
    "37: surfboard",
    "38: tennis racket",
    "39: bottle",
    "40: wine glass",
    "41: cup",
    "42: fork",
    "43: knife",
    "44: spoon",
    "45: bowl",
    "46: banana",
    "47: apple",
    "48: sandwich",
    "49: orange",
    "50: broccoli",
    "51: carrot",
    "52: hot dog",
    "53: pizza",
    "54: donut",
    "55: cake",
    "56: chair",
    "57: couch",
    "58: potted plant",
    "59: bed",
    "60: dining table",
    "61: toilet",
    "62: TV",
    "63: laptop",
    "64: mouse",
    "65: remote",
    "66: keyboard",
    "67: cell phone",
    "68: microwave",
    "69: oven",
    "70: toaster",
    "71: sink",
    "72: refrigerator",
    "73: book",
    "74: clock",
    "75: vase",
    "76: scissors",
    "77: teddy bear",
    "78: hair drier",
    "79: toothbrush"
]

#Remove previous YAML file
!rm {COCO_DATASET_YAML_PATH}

#Write new YAML file with COCO classes
with open(COCO_DATASET_YAML_PATH, 'a') as file:
    file.write("names:\n")
    for item in COCO_CLASS_list:
        file.write(f"  {item}\n")
    file.write("\ntest: ../images/test\ntrain: ../images/train/\nval: ../images/val/")

In [ ]:
# Define the folder containing the label files
label_folder = COCO_DATASET_DIR + '/labels/val'

# Process each file in the label folder
for filename in os.listdir(label_folder):
    file_path = os.path.join(label_folder, filename)
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Replace numbers in each line
    new_lines = []
    for line in lines:
        components = line.strip().split()
        if components[0] in number_mapping:
              new_value = number_mapping[components[0]]
              if new_value == 'remove':
                    continue  # Skip this line if marked for removal
              else:
                    components[0] = number_mapping[components[0]]
                    new_line = ' '.join(components)
                    new_lines.append(new_line)

    # Write the modified contents back to the file
    with open(file_path, 'w') as file:
        file.write('\n'.join(new_lines) + '\n')

print('Number replacement complete.')

In [ ]:
%cd {HOME}
from ultralytics import YOLOv10

#Get model paths
TRAINED_MODEL_PATH      = f'{OUTPUT_PATH}/weights/best.pt'


#pretrained_model.classes = [18]
trained_model    = YOLOv10(TRAINED_MODEL_PATH)

#Validate trained model with 'test' data
trained_model_metrics = trained_model.val(
    data=DATASET_YAML_PATH,     #Specifies the path to the dataset configuration (yaml) file
    imgsz=640,                  #Defines the size of input images
    batch=16,          	        #Sets the number of images per batch. Use -1 for AutoBatch, which automatically adjusts based on GPU memory availability.
    save_json=True,            #If True, saves the results to a JSON file for further analysis or integration with other tools.
    save_hybrid=False,          #If True, saves a hybrid version of labels that combines original annotations with additional model predictions.
    conf=0.5, 	                #Sets the minimum confidence threshold for detections. Detections with confidence below this threshold are discarded.
    iou=0.6, 	                  #Sets the Intersection Over Union (IoU) threshold for Non-Maximum Suppression (NMS). Helps in reducing duplicate detections.
    max_det=50, 	              #Limits the maximum number of detections per image. Useful in dense scenes to prevent excessive detections.
    half=True, 	                #Enables half-precision (FP16) computation, reducing memory usage and potentially increasing speed with minimal impact on accuracy.
    device=None, 	              #Specifies the device for validation (cpu, cuda:0, etc.). Allows flexibility in utilizing CPU or GPU resources.
    dnn=False, 	                #If True, uses the OpenCV DNN module for ONNX model inference, offering an alternative to PyTorch inference methods.
    plots=True,	                #When set to True, generates and saves plots of predictions versus ground truth for visual evaluation of the model's performance.
    rect=False, 	              #If True, uses rectangular inference for batching, reducing padding and potentially increasing speed and efficiency.
    split='val',               #Determines the dataset split to use for validation (val, test, or train).
)

#Define Models

PRETRAINED_MODEL_PATH   = MODEL_PATH
pretrained_model = YOLOv10(PRETRAINED_MODEL_PATH)

#Validate trained model with 'test' data
pretrained_model_metrics = pretrained_model.val(
    data=COCO_DATASET_YAML_PATH,     #Specifies the path to the dataset configuration (yaml) file
    imgsz=640,                  #Defines the size of input images
    batch=16,          	        #Sets the number of images per batch. Use -1 for AutoBatch, which automatically adjusts based on GPU memory availability.
    save_json=False,            #If True, saves the results to a JSON file for further analysis or integration with other tools.
    save_hybrid=True,          #If True, saves a hybrid version of labels that combines original annotations with additional model predictions.
    conf=0.5, 	                #Sets the minimum confidence threshold for detections. Detections with confidence below this threshold are discarded.
    iou=0.6, 	                  #Sets the Intersection Over Union (IoU) threshold for Non-Maximum Suppression (NMS). Helps in reducing duplicate detections.
    max_det=50, 	              #Limits the maximum number of detections per image. Useful in dense scenes to prevent excessive detections.
    half=True, 	                #Enables half-precision (FP16) computation, reducing memory usage and potentially increasing speed with minimal impact on accuracy.
    device=None, 	              #Specifies the device for validation (cpu, cuda:0, etc.). Allows flexibility in utilizing CPU or GPU resources.
    dnn=False, 	                #If True, uses the OpenCV DNN module for ONNX model inference, offering an alternative to PyTorch inference methods.
    plots=True,	                #When set to True, generates and saves plots of predictions versus ground truth for visual evaluation of the model's performance.
    rect=False, 	              #If True, uses rectangular inference for batching, reducing padding and potentially increasing speed and efficiency.
    split='val',               #Determines the dataset split to use for validation (val, test, or train).
    #classes=[17,18,19],         #17 - Horse, 18- Sheep, 19- Cow
)

**NOTE:** Let's randomly select an image from our validation set and visualize the results.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_metrics_comparison(title,model1_name,model1_metrics,model2_name, model2_metrics,min,max):
    metrics = list(model2_metrics.keys())
    model2_values = [model2_metrics[metric] for metric in metrics]
    model1_values = []
    for metric in metrics :
      if str(metric) in model1_metrics:
          model1_values.append(model1_metrics[metric])
      else:
          model1_values.append(0)
    #model2_values = [model2_metrics[metric] for metric in metrics]
    x = range(len(metrics))

    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35

    bars1 = ax.bar(x, model1_values, width=bar_width, label=model1_name, align='center')
    bars2 = ax.bar([p + bar_width for p in x], model2_values, width=bar_width, label=model2_name, align='center')

    # Add text annotations for Model 1
    for bar in bars1:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.01, round(yval, 2), ha='center', va='bottom')

    # Add text annotations for Model 2
    for bar in bars2:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.01, round(yval, 2), ha='center', va='bottom')

    ax.set_xlabel('Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.set_xticks([p + bar_width / 2 for p in x])
    ax.set_xticklabels(metrics)
    ax.set_ylim(min, max)  # All metrics values range between 0 and 1
    ax.legend()

    plt.tight_layout()
    plt.show()

#Pretrained Model Metric Allocation
PT_box = pretrained_model_metrics.box
PT_box_metrics = {
    'Precision'   : PT_box.mp,
    'Recall'      : PT_box.mr,
    'mAP50'       : PT_box.map50,
    'mAP75'       : PT_box.map75,
    'mAP50-95'    : PT_box.map,
    'Fitness'     : PT_box.fitness(),
}

PT_speed = pretrained_model_metrics.speed
PT_speed_metrics ={
    'Preprocess'  : PT_speed['preprocess'],
    'Inference'   : PT_speed['inference'],
    'Postprocess' : PT_speed['postprocess'],
}

#Trained Model Metric Allocation
TR_box = trained_model_metrics.box
TR_speed = trained_model_metrics.speed
TR_box_metrics = {
    'Precision'   : TR_box.mp,
    'Recall'      : TR_box.mr,
    'mAP50'       : TR_box.map50,
    'mAP75'       : TR_box.map75,
    'mAP50-95'    : TR_box.map,
    'Fitness'     : TR_box.fitness(),
}
TR_speed_metrics ={
    'Preprocess'  : TR_speed['preprocess'],
    'Inference'   : TR_speed['inference'],
    'Postprocess' : TR_speed['postprocess'],
}

plot_metrics_comparison('Model Performance' ,'Pre-trained Model',PT_box_metrics,'Custom Model',TR_box_metrics,0,1.05)
plot_metrics_comparison('Runtime Performance','Pre-trained Model',PT_speed_metrics,'Custom Model',TR_speed_metrics,0,50)

In [ ]:
class ResultSummary:
    # Constructor to initialize the object
    def __init__(self,val):
        self.Precision = {}
        self.Recall = {}
        self.F1 = {}
        self.mAP50 = {}
        self.mAP5095 = {}

        for i, c in enumerate(val.box.ap_class_index):
            self.Precision[val.names[c].capitalize()]    = val.box.p[i]      #val.box.class_result(i)[0]
            self.Recall[val.names[c].capitalize()]       = val.box.r[i]      #val.box.class_result(i)[1]
            self.F1[val.names[c].capitalize()]           = val.box.f1[i]
            self.mAP50[val.names[c].capitalize()]        = val.box.ap50[i]   #val.box.class_result(i)[2]
            self.mAP5095[val.names[c].capitalize()]      = val.box.ap[i]     #val.box.class_result(i)[3]

PT_result = ResultSummary(pretrained_model_metrics)
TR_result = ResultSummary(trained_model_metrics)

#Per Class Metrics
plot_metrics_comparison('Precision','Pre-trained Model',PT_result.Precision,'Trained Model',TR_result.Precision,0,1)
plot_metrics_comparison('Recall','Pre-trained Model',PT_result.Recall,'Trained Model',TR_result.Recall,0,1)
plot_metrics_comparison('F1','Pre-trained Model',PT_result.F1,'Trained Model',TR_result.F1,0,1)
plot_metrics_comparison('mAP50','Pre-trained Model',PT_result.mAP50,'Trained Model',TR_result.mAP50,0,1)
plot_metrics_comparison('mAP5095','Pre-trained Model',PT_result.mAP5095,'Trained Model',TR_result.mAP5095,0,1)